**<mark>Parameters</mark>**

In [ ]:
#export purview assets matching this keyword
search_keyword = 'mssql://azdemosrv01.database.windows.net/AdventureWorksSimple/dbo/Product'

#asset level details are exported by default, if you want schema level details set the following to True
"""
IMPORTANT -- PLEASE READ.
Depending on the number of assets, schema extraction can be an intensive process potentially taking a significant amount of time.
It is recommended to use a keyword initially to limit the scope of assets for estimation purposes.
"""
extract_schema_level_details = True

**Run Setup**

In [ ]:
%run ./Setup.ipynb

```
Search all (*) Purview assets and create a csv file with asset details
```

**Crawl assets and extract schema information  
This will be taking a longer time to complete.  
If you don't need schema information, you can skip this section**

In [ ]:
#you can change the keyword to match your scope
cnt, dfAssets = searchPurviewAssets(keyword=search_keyword, recursive_read=True)

dfFinalAssets = pd.DataFrame(columns = ['name', 'qualifiedName', 'classification', 'term', 'description','entityType', 'id', 'certifiedBy'])

for index, row in dfAssets.iterrows():

    thisAssetId = dfAssets.loc[index]['id']
    bulk_output = !pv entity readBulk --guid {thisAssetId}
    bulk_json = json.loads(''.join(bulk_output))
    if len(bulk_json['entities'])!=1:
        continue;
    else:
        parentRow = extractAssetRowFromEntity(bulk_json)
        dfFinalAssets = pd.concat([dfFinalAssets, pd.DataFrame([parentRow])], ignore_index=True)
    
    
    if extract_schema_level_details == True:
        cols, refEntities = getColumnStructure(bulk_json)

        if cols == None:
            continue;

        for aCol in cols:
            colId = aCol['guid']
            #find column in referenced entites section
            thisRefEntity = refEntities[colId]
            
            strClassifications = None
            strTerms = None
                
            if 'classifications' in thisRefEntity:
                strClassifications = separator_char.join([aClass['typeName'] for aClass in thisRefEntity['classifications']])

            if 'meanings' in thisRefEntity['relationshipAttributes']:
                thisTermsGuidList = [aTerm['guid'] for aTerm in thisRefEntity['relationshipAttributes']['meanings']]
                thisTermsList = guidToFormalName(thisTermsGuidList)  
                strTerms = separator_char.join(thisTermsList)


            this_col_row = {'name':thisRefEntity['attributes']['name']
            ,'qualifiedName':thisRefEntity['attributes']['qualifiedName']
            ,'classification':strClassifications
            ,'term':strTerms
            ,'description':coalesce([thisRefEntity['attributes']['userDescription'], thisRefEntity['attributes']['description']])
            ,'entityType':thisRefEntity['typeName']
            ,'id':colId}

            dfFinalAssets = pd.concat([dfFinalAssets, pd.DataFrame([this_col_row])], ignore_index=True)
    
    
#save the file to root working folder
if extract_schema_level_details == True:
    export_file_name = os.path.join(root_working_folder, asset_detail_export_file_name_template.format(os.getenv('PURVIEW_NAME')))
else:
    export_file_name = os.path.join(root_working_folder, asset_export_file_name_template.format(os.getenv('PURVIEW_NAME')))
                       
if os.path.exists(export_file_name):
    os.remove(export_file_name)

dfFinalAssets.to_csv(export_file_name, index_label='index')

In [ ]:
print(str(len(dfAssets)) + " asset(s) were extracted from " + os.getenv('PURVIEW_NAME') )

if extract_schema_level_details == True:
    print(str(len(dfFinalAssets)) + " asset(s) with details were extracted from " + os.getenv('PURVIEW_NAME') )